In [1]:
import sys
import os
import numpy as np 
from keras.utils import to_categorical
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
# from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
import keras.backend as K
from sklearn.metrics import f1_score
from keras.callbacks import ModelCheckpoint, Callback
from keras_contrib.layers import CRF
from numpy import newaxis
import sklearn
import subprocess
import fastText
import pickle

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

In [2]:
def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    return caseLookup[casing]


In [3]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [37]:
def get_sentences_germeval(path):
    sentences = []
    with open(path, 'r', encoding = 'UTF-8') as f:
        sentence = []
        for line in f:
            
            line = line.strip()
            
            # append sentence
            if len(line) == 0:
                if len(sentence):
                    sentences.append(sentence)
                sentence = []
                continue
            
            # get sentence tokens
            splits = line.split()
            if splits[0] == '#':
                continue
            temp = [splits[1], splits[3], splits[2]]
            sentence.append(temp)
        
        # append last
        if len(sentence):
            sentences.append(sentence)    
    return sentences

In [5]:
# preproecessing data from Conll
def get_sentences_conll(filename):
    '''
        -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O
    
    '''
    
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,'rb')
    sentences = []
    sentence = []
    for line in f:
        splits = line.split()
        try:
            word=splits[0].decode()
            if word=='-DOCSTART-':
                continue
            label=splits[-1].decode()
            temp=[word,label]
            sentence.append(temp)
        except Exception as e:
            if len(sentence)!=0:
                sentences.append(sentence)
                sentence=[]
    return sentences



In [38]:
trainSentences = get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
devSentences = get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
testSentences = get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

# trainSentences = get_sentences('../data/CONLL/deu/deu_utf.train')
# devSentences = get_sentences('../data/CONLL/deu/deu_utf.testa')
# testSentences = get_sentences('../data/CONLL/deu/deu_utf.testb')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


24000
2200
5100


In [39]:
print(testSentences[0])

[['1951', 'O', 'O'], ['bis', 'O', 'O'], ['1953', 'O', 'O'], ['wurde', 'O', 'O'], ['der', 'O', 'O'], ['nördliche', 'O', 'O'], ['Teil', 'O', 'O'], ['als', 'O', 'O'], ['Jugendburg', 'O', 'O'], ['des', 'O', 'O'], ['Kolpingwerkes', 'O', 'B-OTH'], ['gebaut', 'O', 'O'], ['.', 'O', 'O']]


In [40]:
labelSet = set()
characters= set()
labelL1Set = set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label, L1 in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
            labelL1Set.add(L1)

In [41]:
print(len(labelSet))

18


In [42]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)
    
labelL12Idx = {}
for label in labelL1Set:
    labelL12Idx[label] = len(labelL12Idx)

In [49]:
print(label2Idx)
print(labelL12Idx)
L1embeddings = np.identity(len(labelL12Idx), dtype='float32')
print(L1embeddings)

{'O': 0, 'B-ORGpart': 2, 'B-LOCderiv': 14, 'I-ORG': 3, 'B-PER': 4, 'B-ORG': 5, 'B-ORGderiv': 6, 'B-OTHderiv': 7, 'B-OTH': 8, 'B-PERderiv': 9, 'B-OTHpart': 10, 'B-LOCpart': 1, 'I-OTH': 11, 'I-PER': 12, 'I-LOC': 13, 'B-LOC': 16, 'I-LOCderiv': 17, 'B-PERpart': 15}
{'B-LOCpart': 11, 'B-ORGpart': 0, 'B-OTHpart': 12, 'I-OTH': 1, 'I-PER': 13, 'I-LOC': 14, 'I-OTHpart': 10, 'I-ORGpart': 2, 'O': 15, 'I-OTHderiv': 3, 'I-ORG': 16, 'B-PER': 4, 'I-PERpart': 17, 'B-ORG': 18, 'B-ORGderiv': 19, 'B-OTHderiv': 5, 'B-OTH': 20, 'B-PERderiv': 6, 'I-PERderiv': 21, 'I-ORGderiv': 7, 'B-PERpart': 8, 'B-LOCderiv': 22, 'B-LOC': 9, 'I-LOCderiv': 23, 'I-LOCpart': 24}
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.

In [12]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [13]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'other': 4, 'PADDING_TOKEN': 7, 'contains_digit': 6, 'allLower': 1, 'initialUpper': 3, 'numeric': 0, 'allUpper': 2, 'mainly_numeric': 5}


In [14]:
print(type(trainSentences))

<class 'list'>


In [15]:
print(trainSentences[0])

[['Schartau', 'O'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'O'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'O'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [16]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
char2Idx['UNKNOWN'] = len(char2Idx)
print(char2Idx)

{'ʻ': 0, 'Æ': 1, '冲': 256, 'p': 2, '›': 276, 'Â': 274, '[': 3, 'У': 4, 'أ': 216, '7': 6, 'Á': 7, 'з': 8, 'Î': 327, 'т': 9, '≘': 271, 'õ': 12, 'ť': 104, '~': 13, 'ń': 15, 'ő': 17, '0': 16, 'λ': 108, 'Л': 19, 'ế': 20, '<': 21, 'Ş': 242, 'ą': 22, '/': 23, 'H': 25, 'й': 26, '柯': 55, '¸': 27, 'έ': 32, '½': 31, 'e': 30, 'v': 33, 'а': 35, 'ا': 36, 'F': 37, '\x95': 38, '“': 39, '(': 325, '\x92': 41, '±': 42, 'X': 43, 'о': 308, '‘': 226, '#': 45, 'б': 116, 'w': 46, '鷹': 5, '=': 263, 'オ': 47, 'G': 277, 'Ш': 48, 'ē': 49, 'г': 212, 'κ': 50, 'ā': 52, 'I': 53, 'q': 54, '太': 56, 'ῦ': 57, '⊃': 285, '+': 58, 'ř': 60, 'ς': 62, 'S': 61, '-': 117, 'М': 282, '守': 65, '´': 64, '傳': 70, 'ę': 67, '–': 68, 'ラ': 69, '章': 71, 'Ġ': 273, '별': 73, '\x94': 74, 'л': 75, 'g': 76, '—': 324, 'ø': 230, 'á': 281, 'n': 77, 'є': 78, "'": 79, 'Ö': 80, '九': 292, 'ρ': 82, 'İ': 83, '9': 259, '\u200e': 10, 'ъ': 85, 'ú': 290, '‚': 86, '°': 87, '佐': 126, '別': 93, 'c': 90, 'В': 91, 'Т': 11, '南': 94, 'ň': 95, 'Q': 63, '>': 101, '`':

In [17]:
print(trainSentences[0]) 

[['Schartau', 'O'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'O'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'O'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [18]:
ft = fastText.load_model("../embeddings/wiki.de.bin")
# ft = fastText.load_model("../embeddings/cc.de.300.bin")

nb_embedding_dims = ft.get_dimension()

In [19]:
print(nb_embedding_dims)
print(len(trainSentences[0]))

300
25


In [20]:
print(trainSentences[0])

[['Schartau', 'O'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'O'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'O'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [21]:
def createBatches(dataset):
    l = []
    for i in dataset:
        l.append(len(i))
    l = set(l)
    print(len(l))
    batches = []
    batch_len = []
    z = 0
    for i in l:
        temp = []
        for batch in dataset:
            if len(batch) == i:
                temp.append(batch)
                z += 1
        batches.append(temp)
#         batch_len.append(z)
    return batches

In [59]:
train_batches = createBatches(trainSentences)
dev_batches = createBatches(devSentences)
test_batches = createBatches(testSentences)

51
45
47


In [71]:
batches = train_batches
print(len(batches))
print(len(batches[0]))
print(batches[0])


51
3
[[['Alles', 'O', 'O'], ['richtig', 'O', 'O'], ['.', 'O', 'O']], [['Farben', 'O', 'O'], ['eingeführt', 'O', 'O'], ['.', 'O', 'O']], [['Material', 'O', 'O'], ['gewinnen', 'O', 'O'], ['.', 'O', 'O']]]


In [81]:
print(getCasing(".", case2Idx))
print(L1embeddings[1])
for input_data, output_data in generator(train_batches):
    print(input_data[1])
    break

4
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
   0. 0.]]]


In [82]:
def generator(batches: 'list of training/dev sentences- batches already created'):
    global line_number
    
    while True:
        for batch in batches:
            word_embeddings = []
            case_embeddings = []
            char_embeddings = []
            nerlevel1_embeddings = []

            output_labels = []
            for index in range(len(batch)): # batches made according to the size of the sentences. len(batch) gives the size of current batch
                sentence = batch[index]
    #             print(sentence)
                temp_casing = []
                temp_char=[]
                temp_word=[]
                temp_output=[]
                temp_nerlevel1_embeddings=[]
                for word in sentence:
                    word, label, L1 = word
                    casing = getCasing(word, case2Idx)
                    temp_casing.append(casing)
                    temp_char2=[]
                    for char in word:
                        if char in char2Idx.keys():
                            temp_char2.append(char2Idx[char])
                        else:
                            temp_char2.append(char2Idx['UNKNOWN']) # To incorporate the words which are not in the vocab
                    temp_char2 = np.array(temp_char2)
                    temp_char.append(temp_char2)
                    word_vector = ft.get_word_vector(word.lower())
                    # word_vector = ft.get_word_vector(word)
                    temp_word.append(word_vector)
                    temp_output.append(label2Idx[label])
                    temp_nerlevel1_embeddings.append(labelL12Idx[L1])
                temp_char = pad_sequences(temp_char, 52)
                word_embeddings.append(temp_word)
                case_embeddings.append(temp_casing)
                char_embeddings.append(temp_char)
                nerlevel1_embeddings.append(temp_nerlevel1_embeddings)
                
                temp_output = to_categorical(temp_output, len(label2Idx))
                output_labels.append(temp_output)
    #             output_labels = to_categorical()
    #             output_labels = np.array(output_labels)
    #             output_labels = output_labels[...,newaxis]

    #             print(np.array(word_embeddings).shape)
    #             print(np.array(case_embeddings).shape)
    #             print(np.array(char_embeddings).shape)
    #             print(output_labels.shape)
    #             print("******************\n\n")
            yield ([np.array(word_embeddings), np.array(nerlevel1_embeddings), np.array(case_embeddings), np.array(char_embeddings)], np.array(output_labels))

def get_label_from_categorical(a):
    labels = []
    for label in a:
        label = np.ndarray.tolist(label)
        label = np.argmax(label)
        labels.append(label)
    return(labels)

def predict_batches(batch):
    steps = 0
    true_labels = []
    pred_labels = []
    for input_data, output_data in generator(batch):
        pred_labels_batch = model.predict(input_data)
        for s in pred_labels_batch:
            pred_labels.append(get_label_from_categorical(s))
        for s in output_data:
            true_labels.append(get_label_from_categorical(s))
        steps += 1
        if steps == len(batch):
            break
    return(true_labels, pred_labels)

In [66]:
idx2Label = {v: k for k, v in label2Idx.items()}
idx2LabelL1 = {v: k for k, v in labelL12Idx.items()}

In [67]:
def f1_score(y_true, y_pred):
    true_labels = []
    pred_labels = []
    for s in y_pred:
        pred_labels.append(get_label_from_categorical(s))
    for s in y_true:
        true_labels.append(get_label_from_categorical(s))
    p, r, f = compute_f1(y_pred, y_true, idx2Label)
    return r

def get_model():
    words_input = Input(shape=(None, nb_embedding_dims), dtype='float32', name='words_input')
    L1_input = Input(shape=(None,), dtype='int32', name='L1_input')
    L1_embed = Embedding(output_dim=L1embeddings.shape[1], input_dim=L1embeddings.shape[0], weights=[L1embeddings], trainable=False, name = 'L1_embed')(L1_input)
    casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
    casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False, name = 'case_embed')(casing_input)
    character_input=Input(shape=(None,52,),name='char_input')
    embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
    kernel_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in kernel_sizes:
        conv = TimeDistributed(Conv1D(
                             kernel_size=sz,
                             filters=32,
                             padding="same",
                             activation="relu",
                             strides=1))(embed_char_out)
        conv = TimeDistributed(MaxPooling1D(52))(conv)
        conv = TimeDistributed(Flatten())(conv)
        conv_blocks.append(conv)
    output = concatenate([words_input, L1_embed, casing, conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
    output = TimeDistributed(Dense(len(label2Idx)))(output)
    crf = CRF(len(label2Idx))
    output = crf(output)
    model = Model(inputs=[words_input, L1_input, casing_input, character_input], outputs=[output])
    model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
    model.summary()
    return(model)

class F1History(Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.f1_scores = []
        self.max_f1 = 0

    def on_epoch_end(self, epoch, logs={}):
        self.acc.append(logs.get('val_acc'))
        true_labels, pred_labels = predict_batches(dev_batches)
        pre, rec, f1 = compute_f1(pred_labels, true_labels, idx2Label)
        self.f1_scores.append(f1)
        if epoch > 30 and f1 > self.max_f1:
            print("\nNew maximum F1 score: " + str(f1) + " (before: " + str(self.max_f1) + ") Saving to " + tmp_model_filename)
            self.max_f1 = f1
            model.save(tmp_model_filename)

In [83]:
tmp_model_filename = 'tmp_generator_NER_best.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = F1History()
model = get_model()
model.fit_generator(
    generator(train_batches), 
    epochs = 80, steps_per_epoch = len(train_batches), 
    validation_data = generator(dev_batches), validation_steps = len(dev_batches), 
    callbacks = [history]
)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10528       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_90 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_93 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

Epoch 72/80
51/51 [==============================] - 22s 429ms/step - loss: -0.2852 - acc: 0.9995 - val_loss: -0.2966 - val_acc: 0.9969
Epoch 73/80
51/51 [==============================] - 23s 441ms/step - loss: -0.2893 - acc: 0.9995 - val_loss: -0.3011 - val_acc: 0.9974
Epoch 74/80
51/51 [==============================] - 22s 436ms/step - loss: -0.2936 - acc: 0.9994 - val_loss: -0.3063 - val_acc: 0.9971
Epoch 75/80
51/51 [==============================] - 23s 450ms/step - loss: -0.2980 - acc: 0.9995 - val_loss: -0.3100 - val_acc: 0.9971
Epoch 76/80
51/51 [==============================] - 23s 446ms/step - loss: -0.3020 - acc: 0.9993 - val_loss: -0.3112 - val_acc: 0.9971
Epoch 77/80
51/51 [==============================] - 22s 435ms/step - loss: -0.3049 - acc: 0.9991 - val_loss: -0.3189 - val_acc: 0.9972
Epoch 78/80
50/51 [============================>.] - ETA: 0s - loss: -0.3143 - acc: 0.9994
New maximum F1 score: 0.7233009708737864 (before: 0.7135922330097088) Saving to tmp_generator

In [32]:
print(history.acc)
print(history.f1_scores)

[0.9946695905111053, 0.9946695905111053, 0.9753653725981712, 0.971133625805378, 0.9892300726066936, 0.9916527701778846, 0.9934801145033403, 0.9193957837332378, 0.9945173840360207, 0.9694857255708088, 0.9949436785416169, 0.9296631378206339, 0.9950203010981733, 0.9947294317050414, 0.9950850945711136, 0.9952510368282145, 0.9952144384113225, 0.9952569980783896, 0.9953162610530853, 0.9944079532135617, 0.995281142429872, 0.9953866038810123, 0.9953606687350707, 0.9954716947945681, 0.9955632332509214, 0.995618404041637, 0.9956095008416609, 0.9956665044752034, 0.9953894490545446, 0.9953945659778335, 0.995706004283645, 0.9957643909887834, 0.9956305682930079, 0.9957253030213443, 0.9958990923112089, 0.9957777957211841, 0.9955365413427353, 0.9957157587734136, 0.9956198622421785, 0.9956063252958385, 0.9958345579559152, 0.9956996938586236, 0.9958652172847228, 0.9959894129092043, 0.9959836213155226, 0.9959905016151341, 0.9959326248277317, 0.9959960159659386, 0.9962591779232025, 0.9959572835402055, 0.9

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
true_labels, pred_labels = predict_batches(test_batches)
print(compute_f1(pred_labels, true_labels, idx2Label))

# Experiments

In [ ]:
f = open('results_wiki.txt', 'w')
for run_i in range(13):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'tmp_generator_NER_best.' + str(run_i) + '.h5'
    # tmp_model_filename = 'tmp_generator_NER_best.h5'
    # checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
    history = F1History()
    
    model = get_model()
    model.fit_generator(
        generator(train_batches), 
        epochs = 80, steps_per_epoch = len(train_batches), 
        validation_data = generator(dev_batches), validation_steps = len(dev_batches), 
        callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    true_labels, pred_labels = predict_batches(test_batches)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_generator_NER_best.0.h5', 'final_model_germeval.h5')
with open("final_model_germeval.indexes", "w") as f:
    json.dump([idx2Label, label2Idx, char2Idx, case2Idx], f)